In [1]:
import numpy as np
from os import listdir, stat
import re
from IPython.display import HTML, display
import matplotlib.pyplot as plt

In [2]:
re_app_sent = re.compile(r'^#APP: sent (\d+)\s*bts\.$')
re_cli_sent = re.compile(r'^#CLI: sent (\d+)\s*bts\.$')
re_cli_used = re.compile(r'^#CLI: used (\d+)\s*bts\.$')
re_cli_took = re.compile(r'^#CLI: took ((\d+)\s*ms\.)?((\d+)\s*ns\.)?$')
re_log_num = re.compile(r'^\D+(\d+)\.log$')
re_bin_num = re.compile(r'^\D+(\d+)\.bin$')

# Returns an array with columns [time, network, memory]
def readLogs(path):
    logs = listdir(path)
    data = np.zeros((len(logs), 3))
    for log in logs:
        filename = path+'/'+log
        with open(filename,'r') as file:
            log_num = int(re_log_num.match(log).group(1))-1
            found_christmas = False
            app_sent_n = -1
            cli_sent_n = -1
            cli_used_n = -1
            cli_took_n = -1
            while True:
                line = file.readline()
                if not line:
                    break
                elif line.startswith('#CLI: What are you waitin for? Christmas?'):
                    found_christmas = True
                else:
                    app_sent = re_app_sent.match(line)
                    cli_sent = re_cli_sent.match(line)
                    cli_used = re_cli_used.match(line)
                    cli_took = re_cli_took.match(line)
                    if (app_sent):
                        app_sent_n = int(app_sent.group(1))
                    elif (cli_sent):
                        cli_sent_n = int(cli_sent.group(1))
                    elif (cli_used):
                        cli_used_n = int(cli_used.group(1))
                    elif (cli_took):
                        if (cli_took.group(4)):
                            cli_took_n = int(cli_took.group(4))
                        else:
                            cli_took_n = int(cli_took.group(2))*1000000
                #print(line)
            if not found_christmas:
                print('ERROR! Did not find Christmas! In '+filename)
            elif app_sent_n < 0 or cli_sent_n < 0 or cli_used_n < 0 or cli_took_n < 0:
                print('ERROR! Missing some stats! In '+filename)
            else:
                data[log_num, 0] = cli_took_n
                data[log_num, 1] = cli_sent_n+app_sent_n
                data[log_num, 2] = cli_used_n
    return data


def readLanguage(path):
    return {
        'base': readLogs(path+'/base'),
        'static': readLogs(path+'/static'),
        'dynamic': readLogs(path+'/dynamic')
    }

def sizeBins(path):
    bins = listdir(path)
    data = np.zeros((len(bins), 1))
    for binfile in bins:
        filename = path+'/'+binfile
        bin_num = 0
        if (len(bins) > 1):
            bin_num = int(re_bin_num.match(binfile).group(1))-1
        data[bin_num] = stat(filename).st_size
    return data

def sizeLanguage(path):
    sizes = {
        'static': sizeBins(path+'/static'),
        'dynamic': sizeBins(path+'/dynamic')
    }
    sizes['base'] = np.repeat(sizeBins(path+'/base'),sizes['static'].shape[0],axis=0);
    return sizes

def readLanguages(logdir, bindir, languages):
    data = {}
    for language in languages:
        data[language] = {}
        stats = readLanguage(logdir+'/'+language)
        sizes = sizeLanguage(bindir+'/'+language)
        for mode, array in stats.items():
            data[language][mode] = np.concatenate((array, sizes[mode]), axis=1)
    return data

In [3]:
# Calculate statistics on all data
def calculateStats(data, scaling):
    stats = {}
    for language, modes in data.items():
        stats[language] = {}
        for mode, array in modes.items():
            scaled = array/scaling
            stats[language][mode] = {
                'mean': np.mean(scaled, axis=0),
                'std': np.std(scaled, axis=0)
            }
    return stats

def calculateDiffStats(data, scaling):
    stats = {}
    for language, modes in data.items():
        stats[language] = {}
        base_mean_per_msg = np.mean(modes['base'], axis=0)/scaling
        for mode, array in modes.items():
            if mode != 'base':
                mode_diff_per_msg = array/scaling - base_mean_per_msg
                stats[language][mode] = {
                    'mean': np.mean(mode_diff_per_msg, axis=0),
                    'std': np.std(mode_diff_per_msg, axis=0)
                }
    return stats

In [4]:
def formatNumber(number, unit):
    order = ''
    if number > 1e9:
        number = number/1e9
        order = 'G'
    elif number > 1e6:
        number = number/1e6
        order = 'M'
    elif number > 1e3:
        number = number/1e3
        order = 'k'
    elif number < 1e-6:
        number = number*1e9
        order = 'n'
    elif number < 1e-3:
        number = number*1e6
        order = 'µ'
    elif number < 1:
        number = number*1e3
        order = 'm'
        
    return ('%.2f'%number)+order+unit

def tableData(data, i, scale, unit):
    return '<td>'+formatNumber(data['mean'][i]*scale, unit)+' ± '+formatNumber(data['std'][i]*scale, unit)+'</td>'

# Display a nice table
def displayStatsTable(stats):
    html = '<table>'
    # Make headers
    html += '<tr><th>Language:</th><th>Mode:</th><th>Time/msg:</th><th>Network/msg:</th><th>Memory:</th><th>Size:</th></tr>'
    # Make rows
    for language, modes in stats.items():
        for mode, data in modes.items():
            html += '<tr><td>'+language+'</td><td>'+mode+'</td>'
            html += tableData(data, 0, 1e-9, 's')
            html += tableData(data, 1, 1, 'b')
            html += tableData(data, 2, 1, 'b')
            html += tableData(data, 3, 1, 'b')
            html += '</tr>'
    # End table
    html += '</table>'
    display(HTML(html))

In [5]:
logdir = '../../../target/thingml-logs'
bindir = '../../../target/thingml-bins'
languages = ['nodejs', 'go', 'arduino']

data = readLanguages(logdir, bindir, languages)

stats = calculateStats(data, np.array([300, 300, 1, 1]))
diffstats = calculateDiffStats(data, np.array([300, 300, 1, 1]))

displayStatsTable(stats)
displayStatsTable(diffstats)

Language:,Mode:,Time/msg:,Network/msg:,Memory:,Size:
nodejs,base,61.87µs ± 9.40µs,4.33b ± 0.00nb,3.05Mb ± 44.71kb,26.03kb ± 0.00nb
nodejs,static,110.10µs ± 11.97µs,7.30b ± 862.93mb,3.20Mb ± 27.75kb,32.50kb ± 16.01b
nodejs,dynamic,141.03µs ± 19.60µs,7.44b ± 896.53mb,3.35Mb ± 56.77kb,44.49kb ± 20.39b
go,base,2.72µs ± 2.42µs,4.33b ± 0.00nb,116.59kb ± 267.33b,2.16Mb ± 0.00nb
go,static,4.46µs ± 1.65µs,7.30b ± 862.93mb,135.66kb ± 2.41kb,2.27Mb ± 3.84kb
go,dynamic,4.66µs ± 1.76µs,7.44b ± 896.53mb,136.92kb ± 2.17kb,2.39Mb ± 5.81kb
arduino,base,463.33µs ± 0.00ns,4.33b ± 0.00nb,697.00b ± 0.00nb,13.44kb ± 0.00nb
arduino,static,1.27ms ± 74.13µs,7.30b ± 862.93mb,773.69b ± 8.39b,21.53kb ± 395.38b
arduino,dynamic,1.50ms ± 76.81µs,7.44b ± 896.53mb,843.63b ± 13.72b,37.40kb ± 1.01kb


Language:,Mode:,Time/msg:,Network/msg:,Memory:,Size:
nodejs,static,48.23µs ± 11.97µs,2.97b ± 862.93mb,148.40kb ± 27.75kb,6.47kb ± 16.01b
nodejs,dynamic,79.17µs ± 19.60µs,3.10b ± 896.53mb,295.08kb ± 56.77kb,18.46kb ± 20.39b
go,static,1.75µs ± 1.65µs,2.97b ± 862.93mb,19.07kb ± 2.41kb,108.39kb ± 3.84kb
go,dynamic,1.94µs ± 1.76µs,3.10b ± 896.53mb,20.34kb ± 2.17kb,227.30kb ± 5.81kb
arduino,static,809.90µs ± 74.13µs,2.97b ± 862.93mb,76.69b ± 8.39b,8.09kb ± 395.38b
arduino,dynamic,1.04ms ± 76.81µs,3.10b ± 896.53mb,146.63b ± 13.72b,23.96kb ± 1.01kb
